# काट-छाँट और परिमाणीकरण संयुक्त

<!-- ## पूर्वापेक्षाएँ

1. आभासी वातावरण बनाने की अनुशंसा की जाती है

2. बुनियादी पर्यावरण निर्भरताएँ कॉन्फ़िगर करें

3. यदि आपके पास CUDA-समर्थित GPU है, तो --> जोड़ें

## तैयारी

1. **CIFAR-10** डेटासेट का उपयोग करें
>CIFAR-10 डेटासेट मशीन लर्निंग और कंप्यूटर विज़न में व्यापक रूप से उपयोग किया जाने वाला बेंचमार्क है, जिसमें 60,000 32x32 रंगीन छवियां शामिल हैं जो 10 श्रेणियों में विभाजित हैं, प्रत्येक श्रेणी में 6,000 छवियां हैं।

2. **ResNet18** पूर्व-प्रशिक्षित मॉडल का उपयोग करें
> अवशिष्ट तंत्रिकाअल नेटवर्क (जिसे अवशिष्ट नेटवर्क या रेसनेट के रूप में भी जाना जाता है) एक अग्रणी गहन शिक्षण मॉडल है जिसमें वजन परत अवशिष्ट फ़ंक्शन को सीखने के लिए परत इनपुट को संदर्भित करता है। इसे 2015 में छवि पहचान के लिए विकसित किया गया था और इसने इमेजनेट लार्ज स्केल विज़ुअल रिकग्निशन चैलेंज जीता था। ILSVRC) उस वर्ष।
ResNet-18 मॉडल संरचना विज़ुअलाइज़ेशन और अन्य जानकारी के लिए, कृपया ली म्यू का [d2l](https://d2l.ai/chapter_convolutional-modern/resnet.html) देखें

In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn.utils.prune as prune
import torch.optim as optim


In [ ]:
# CIFAR-10 को लोड और सामान्य करें
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# डेटासेट को प्रशिक्षण और परीक्षण सेट में विभाजित करें
trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=4,
                                         shuffle=False, num_workers=2)

In [9]:
# ResNet-18 पूर्व-प्रशिक्षित मॉडल पैरामीटर डाउनलोड करें
resnet18 = torchvision.models.resnet18()
# ResNet-18 मॉडल संरचना
resnet18

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [11]:
def prune_model(model, pruning_rate=0.1):
    for name, module in model.named_modules():
        if isinstance(module, torch.nn.Conv2d) or isinstance(module, torch.nn.Linear):
            
# असंरचित एल1 मानक छंटाई का उपयोग करें
            prune.l1_unstructured(module, name='weight', amount=pruning_rate)
            
            prune.remove(module, 'weight')

In [ ]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.SGD(resnet18.parameters(), lr=0.001, momentum=0.9)

def train_model(model, epochs=10, prune_every_n_epochs=5):
    for epoch in range(epochs):
        running_loss = 0.0
        for i, data in enumerate(trainloader, 0):
            inputs, labels = data
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            
            if i % 2000 == 1999:
                print(f'[{epoch + 1}, {i + 1}] loss: {running_loss / 2000}')
                running_loss = 0.0
                
        if (epoch + 1) % prune_every_n_epochs == 0:
            print(f'Pruning after epoch {epoch + 1}')
            prune_model(model, pruning_rate=0.1)
            print('Pruning done.')

train_model(resnet18)

In [ ]:
# मॉडल को int8 प्रकार में परिमाणित करें
resnet18_int8 = torch.quantization.convert(resnet18, inplace=False)

# परिमाणित मॉडल सहेजें
# टॉर्च.सेव(resnet18_int8.state_dict(), 'resnet18_int8.pth')

# परिमाणित मॉडल को लोड करें और उसका मूल्यांकन करें
resnet18_int8_loaded = torchvision.models.resnet18()
resnet18_int8_loaded.qconfig = torch.quantization.get_default_qconfig('fbgemm')
resnet18_int8_loaded = torch.quantization.prepare(resnet18_int8_loaded)
resnet18_int8_loaded = torch.quantization.convert(resnet18_int8_loaded)
resnet18_int8_loaded.load_state_dict(torch.load('resnet18_quantized.pth'))
